In [ ]:
# Python or R code based on the Language property.
#
# Note that a few lines of Python or R code are added before your code; for example:
# Python:
#  dm_class_input = ["class_var_1", "class_var_2"]
#  dm_interval_input = ["numeric_var_1", "numeric_var_2"]
# R:
#  dm_class_input <- c("class_var_1", "class_var_2")
#  dm_interval_input <- c("numeric_var_1", "numeric_var_2")
#
# For Python, use the Node Configuration section of the Project Settings to prepend
# any configuration code, which is executed before the above code. During execution,
# this code is automatically prepended to every node that runs Python code.
#
# After running the node, the Python or R code window in the node results displays
# the actual code that was executed. START ENTERING YOUR CODE ON THE NEXT LINE.
import sklearn
from sklearn.linear_model import LogisticRegression

# Get full data with inputs + partition indicator
dm_input.insert(0, dm_partitionvar)
fullX = dm_inputdf.loc[:, dm_input]

# Dummy encode class variables
fullX_enc =  pd.get_dummies(fullX, columns=dm_class_input, drop_first=True)

# Create X (features/inputs); drop partition indicator
X_enc = fullX_enc[fullX_enc[dm_partitionvar] == dm_partition_train_val]
X_enc = X_enc.drop(dm_partitionvar, axis=1)

# Create y (labels)
y = dm_traindf[dm_dec_target]

#----------
# Training
#----------
# Fit Random Forest model w/ training data
dm_model = LogisticRegression(C = 0.7, random_state=42, solver='liblinear', class_weight='balanced')
dm_model.fit(X_enc, y)
print(dm_model)

#----------
# Scoring
#----------
# Score full data: posterior probabilities
fullX_enc = fullX_enc.drop(dm_partitionvar, axis=1)
dm_scoreddf_prob = pd.DataFrame(dm_model.predict_proba(fullX_enc),
    columns=dm_predictionvar)

# Score full data: class prediction
dm_scoreddf_class = pd.DataFrame(dm_model.predict(fullX_enc),
    columns=[dm_classtarget_intovar])

# Column merge posterior probabilities and class prediction
dm_scoreddf = pd.concat([dm_scoreddf_prob, dm_scoreddf_class], axis=1)
print(dm_scoreddf.head(10))